# 5311 Insights
<hr style="border:2px solid #8CBCCB">


In [1]:
import pandas as pd
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 

#Formatting notebook
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.0f}".format
pd.options.display.float_format = '{:,}'.format
import warnings
warnings.filterwarnings("ignore")

#Import script 
import _data_prep as data_prep
import _utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [45]:
#Color Palette
LIGHT_BLUE ="#8CBCCB"
DARK_BLUE = "#2EA8CE"
YELLOW = "#F4D837"
GREEN = "#51BF9D"
PURPLE = "#9487C0"

In [3]:
#Cleaned & aggregated DF 
aggregated = data_prep.aggregated_df()
final = data_prep.final_df()

## Analyze 5311 by District

In [41]:
district = (aggregated.groupby(['caltrans_district',])
           .agg({
                 'doors_sum':'sum',
                 'allocationamount':'sum',
                 'expendedamount':'sum',
                 'total_vehicles':'sum',
                 'average_age_of_fleet__in_years_':'median'
                 })
            .reset_index()
) 

district = (_utils
            .cols_cleanup(district)
           ) 

In [48]:
(district
 .style.bar(subset=["Allocationamount"], color=LIGHT_BLUE)
 .bar(subset=["Expendedamount"], color=PURPLE)
 .bar(subset=["Total Vehicles"], color=YELLOW)
 .bar(subset=["Doors Sum"], color=GREEN)
)

,Caltrans District,Doors Sum,Allocationamount,Expendedamount,Total Vehicles,Average Age Of Fleet In Years
0,01 - Eureka,193.000000,18514520.000000,12328651.030000,177.000000,5.424279
1,02 - Redding,78.000000,6293412.000000,4625264.020000,62.000000,5.938889
2,03 - Marysville,511.000000,16444169.000000,11525387.780000,353.000000,6.592634
3,04 - Oakland,3044.000000,10945699.280000,7418303.000000,2145.000000,7.201646
4,05 - San Luis Obispo,681.000000,16604962.000000,11601678.880000,531.000000,7.128430
5,06 - Fresno,639.000000,29962524.000000,21368584.630000,507.000000,6.691667
6,07 - Los Angeles,12.000000,2706782.000000,2169754.170000,7.000000,7.500000
7,08 - San Bernardino,1264.000000,17885468.000000,13913177.690000,1076.000000,4.000000
8,09 - Bishop,146.000000,3830444.000000,2467504.560000,112.000000,11.000000
9,10 - Stockton,440.000000,11420717.000000,8459654.400000,335.000000,6.156062


## What are the GTFS statuses of applicants?

In [5]:
GTFS_orgs = (aggregated.groupby(['GTFS'])
             .agg({'organization_name':'nunique'})
             .reset_index()
             .rename(columns = {'organization_name': 'Count_of_Organizations'})
             .sort_values('Count_of_Organizations', ascending = False)
            )

In [6]:
_utils.basic_bar_chart(GTFS_orgs, 'Count_of_Organizations', 'GTFS', 'GTFS') 

alt.Chart(...)

## Which Caltrans District has the most applicants?

In [7]:
Orgs_in_district = (aggregated
                    .groupby(['caltrans_district'])
                    .agg({'organization_name':'nunique'})
                    .reset_index()
                    .rename(columns = {'organization_name':'count_of_organizations'})
                    .sort_values(by='count_of_organizations', ascending=False)
                   )

In [8]:
_utils.basic_bar_chart(Orgs_in_district,'caltrans_district','count_of_organizations','caltrans_district') 

alt.Chart(...)

##  What is the GTFS status by Fleet Size?

In [10]:
vehicle_size=  ['vehicles_percent_older_than_9',
       'vehicles_percent_older_than_15','vehicles_percent_0_to_9']

In [11]:
_utils.multi_charts((_utils.aggregation_one(final, 'GTFS')), 'GTFS', vehicle_size)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Analyze fleet size by vehicle age.

In [12]:
fleet = _utils.aggregation_one(final,'fleet_size')

In [13]:
fleet = _utils.cols_cleanup(fleet
         .loc[fleet['fleet_size'] != 'No Info']) 


In [14]:
fleet[['Fleet Size','Vehicles Older Than 9', 'Vehicles Older Than 15',
       'Vehicles 0 To 9',]].style.background_gradient(cmap = 'BuGn' )

,Fleet Size,Vehicles Older Than 9,Vehicles Older Than 15,Vehicles 0 To 9
0,Large,14216.000000,4768.000000,34058.000000
1,Medium,2340.000000,451.000000,10712.000000
3,Small,282.000000,35.000000,990.000000


## What is the fleet size by agency?

In [15]:
fleet_agencies = (aggregated
                  .groupby(['fleet_size'])
                  .agg({'organization_name':'nunique'})
                  .reset_index()
                  .rename(columns = {'organization_name':'Total Organizations'})
                 )

In [16]:
_utils.basic_bar_chart(fleet_agencies,'fleet_size','Total Organizations','fleet_size') 

alt.Chart(...)

## What are the most common reporter types?

In [17]:
Reporter_type_agg = (aggregated
                     .groupby(['reporter_type'])
                     .agg({'organization_name':'nunique'})
                     .reset_index()
                     .rename(columns = {'organization_name':'Count_of_Agencies'})
                    )

In [18]:
_utils.basic_bar_chart(Reporter_type_agg,'Count_of_Agencies', 'reporter_type', 'reporter_type') 

alt.Chart(...)

## Which organization received the most funds overall? What is their GTFS Status?

In [19]:
Most_Money = (aggregated
              .groupby(['organization_name','GTFS'])
              .agg({'allocationamount':'sum'})
              .rename(columns = {'allocationamount': '5311_funds_received'})
              .reset_index()
              .sort_values('5311_funds_received', ascending = False)
              .head(10)
             )

In [20]:
_utils.basic_bar_chart(Most_Money,'5311_funds_received', 'organization_name','organization_name') 

alt.Chart(...)

In [35]:
most_money_list = Most_Money.organization_name.tolist()
gtfs_most_funded = aggregated[aggregated["organization_name"].isin(most_money_list)]
gtfs_most_funded = _utils.cols_cleanup(gtfs_most_funded[['organization_name','GTFS']]).sort_values('Gtfs')
gtfs_most_funded

,Organization Name,Gtfs
42,Fresno County Rural Transit Agency,Static Incomplete_RT Incomplete
58,Monterey-Salinas Transit,Static Incomplete_RT OK
2,Butte County Association of Governments,Static OK_RT Incomplete
47,Kern Regional Transit,Static OK_RT Incomplete
56,Mendocino Transit Authority,Static OK_RT Incomplete
60,Mountain Area Regional Transit Authority,Static OK_RT Incomplete
79,Transit Joint Powers Authority for Merced County,Static OK_RT Incomplete
45,Humboldt Transit Authority,Static OK_RT OK
50,Lake Transit Authority,Static OK_RT OK
83,Victor Valley Transit Authority,Static OK_RT OK


## Which subset of 5311 is the most popular, using allocation requested as a metric?

In [21]:
bc_funds = (final
            .groupby(['funding_program'])
            .agg({'organization_name':'nunique',
                  'allocationamount':'sum'})
            .rename(columns = {'organization_name':'Count_of_Organizations'
                               ,'allocationamount':'total_sum'})
            .reset_index()
            .sort_values(by =['total_sum'])
             )

In [22]:
_utils.basic_bar_chart(bc_funds,'total_sum', 'funding_program', 'funding_program') 

alt.Chart(...)